Import necessary libraries

In [5]:
from bs4 import BeautifulSoup
import requests
import csv
import pandas as pd

In [6]:
# from Corey Schafer web scraping tutorial
# source = requests.get('http://coreyms.com').text

# soup = BeautifulSoup(source, 'lxml')

# csv_file = open('cms_scrape.csv', 'w')

# csv_writer = csv.writer(csv_file)
# csv_writer.writerow(['headline', 'summary', 'video_link'])

# for article in soup.find_all('article'):
#     headline = article.h2.a.text
#     print(headline)

#     summary = article.find('div', class_='entry-content').p.text
#     print(summary)

#     try:
#         vid_src = article.find('iframe', class_='youtube-player')['src']

#         vid_id = vid_src.split('/')[4]
#         vid_id = vid_id.split('?')[0]

#         yt_link = f'https://youtube.com/watch?v={vid_id}'
#     except Exception as e:
#         yt_link = None

#     print(yt_link)

#     print()

#     csv_writer.writerow([headline, summary, yt_link])

# csv_file.close()

Get that webpage and do some basic actions with it

In [7]:
# idea for web scraping: scrape audible best-selling books
# Put the link to Audible Best-Sellers page here no matter the category
URL = "https://www.audible.com/search?sort=popularity-rank&ref=a_search_l1_catBackAll&pf_rd_p=daf0f1c8-2865-4989-87fb-15115ba5a6d2&pf_rd_r=S4VEHTG7BAZHCPTD9YB7"
URL = "https://www.audible.com/search?crid=31E3J6JGY980O&i=na-audible-us&k=brandon+sanderson&keywords=brandon+sanderson&ref-override=a_search_t1_header_search&sort=pubdate-desc-rank&sprefix=brand%2Cna-audible-us%2C170&url=search-alias%3Dna-audible-us&ref=a_search_c1_sort_1&pf_rd_p=073d8370-97e5-4b7b-be04-aa06cf22d7dd&pf_rd_r=VRAHHY22TC5WB0JFHJFR"
webpage = requests.get(URL)
soup = BeautifulSoup(webpage.text, "html.parser")

In [8]:
# many thanks to https://jovian.ai/pratulofficialthings/audible-business-and-careers-books-2022
book_list = soup.find_all('li', class_='bc-list-item productListItem')


def get_book_names(book_list):
    book_names = []
    for tag in book_list:
        a_tag_name = tag.h3.find_all('a', recursive=False)
        book_name = a_tag_name[0].text.strip()
        book_names.append(book_name)
    return book_names

def get_book_length(book_list):
    book_length = []
    for tag in book_list:
        try:
            len_tag = tag.find('li', class_='bc-list-item runtimeLabel')
            length_tag = len_tag.find('span')
            length = length_tag.text.strip()
            book_length.append(length)
        except AttributeError:
            book_length.append(None)
    # Remove "Length: " from every elem of output list if it's not NoneType
    book_length = [i[8:] if i is not None else i for i in book_length]
    return book_length

def get_author(book_contents):
    written_by=[]
    for tag in book_contents:
        author_tag= tag.find('li', class_='bc-list-item authorLabel')
        try:
            auth_tag = author_tag.find('a')
            author = auth_tag.text.strip()
            written_by.append(author)
        except AttributeError:
            written_by.append(None)
    return written_by

def get_description(book_list):
    description = []
    for tag in book_list:
        about_tag = tag.find('li', class_='bc-list-item subtitle')
        try:
            description_tag = about_tag.find('span').text.strip()
            description.append(description_tag)
        except AttributeError:
            description.append(None)
    return description

def get_language(book_list):
    language_list = []
    for tag in book_list:
        lang_tag = tag.find('li', class_='bc-list-item languageLabel')
        try:
            language_tag = lang_tag.find('span').text.split()
            # Access 'English' in ['Language:', 'English'] of language_tag
            language_itself = language_tag[1]
            language_list.append(language_itself)
        except AttributeError:
            language_list.append(None)
    return language_list

def get_rating(book_list):
    rating = []
    for tag in book_list:
        star_tag = tag.find('li', class_='bc-list-item ratingsLabel')
        try:
            rating_tag = star_tag.find(
                'span', class_='bc-text bc-pub-offscreen').text.strip()
            rating.append(rating_tag)
        except AttributeError:
            rating.append(None)
    return rating

def get_nunber_of_ratings(book_list):
    nunber_of_ratings = []
    for tag in book_list:
        star_tag = tag.find('li', class_='bc-list-item ratingsLabel')
        try:
            rating_tag = star_tag.find(
                'span', class_='bc-text bc-size-small bc-color-secondary').text.strip()
            nunber_of_ratings.append(rating_tag)
        except AttributeError:
            nunber_of_ratings.append(None)
    return nunber_of_ratings

def get_regular_price(book_list):
    regular_price = []
    for tag in book_list:
        buy_tag = tag.find(
            'p', class_='bc-text buybox-regular-price bc-spacing-none bc-spacing-top-none')
        try:
            price_tag = buy_tag.find_all(
                'span', class_='bc-text bc-size-base bc-color-base')
            price = price_tag[1].text.strip()
            regular_price.append(price)
        except AttributeError:
            regular_price.append(None)
    return regular_price

def get_member_price(book_list):
    pass

df = pd.DataFrame(
{
    "Book_Name": get_book_names(book_list),
    "Description": get_description(book_list),
    "Author": get_author(book_list),
    "Rating": get_rating(book_list),
    "Num_of_Ratings": get_nunber_of_ratings(book_list),
    "Regular_Price": get_regular_price(book_list),
    "Audio_Length": get_book_length(book_list)
})

df

,Book_Name,Description,Author,Rating,Num_of_Ratings,Regular_Price,Audio_Length
0,The Shepherd's Crown,"Discworld, Book 41",Terry Pratchett,None,Not rated yet,$34.90,Not Yet Known
1,I Shall Wear Midnight,"Discworld, Book 38",Terry Pratchett,None,Not rated yet,$34.90,Not Yet Known
2,Curso de escritura creativa [Creative Writing ...,None,Brandon Sanderson,None,Not rated yet,$21.73,10 hrs and 48 mins
3,Dark One: Forgotten,None,Brandon Sanderson,None,Not rated yet,$20.99,5 hrs
4,The Lost Metal,A Mistborn Novel,Brandon Sanderson,None,Not rated yet,$31.18,18 hrs and 46 mins
5,The Lost Metal,A Mistborn Novel,Brandon Sanderson,None,Not rated yet,$27.27,18 hrs and 46 mins
6,Bastille vs. the Evil Librarians,None,Brandon Sanderson,None,Not rated yet,$16.35,4 hrs and 51 mins
7,Edgedancer,Stormlight Archive,Brandon Sanderson,4.5 out of 5 stars,134 ratings,$20.99,6 hrs and 24 mins
8,Le Puits de l'ascension,Fils des brumes 2,Brandon Sanderson,None,Not rated yet,$37.82,30 hrs and 37 mins
9,Bastille vs. the Evil Librarians,"Alcatraz Versus the Evil Librarians, Book 6",Brandon Sanderson,4.5 out of 5 stars,366 ratings,$17.49,4 hrs and 52 mins
